# Why 'evaluate' is the feature I am missing the most from commercial MIP solvers
If you use optimization out of the box in Python with `scipy`, then you are going to see something along these lines:

In [2]:
import numpy as np
from scipy.optimize import basinhopping

func = lambda x: np.cos(14.5 * x - 0.3) + (x + 0.2) * x
x0=[1.]
minimizer_kwargs = {"method": "BFGS"}
ret = basinhopping(func, x0, minimizer_kwargs=minimizer_kwargs, niter=200)

Now, let's say you would like to check what the objective function evaluates to. Then, you could simply write:

In [4]:
func(5)

25.001603108415402

This direct query makes testing and using across algorithms super easy, as you can simply check out `func` directly. However, what about commerical MIP solvers? Can we do the same?

In [12]:
import xpress as xp

# Define the model and variables
Model = xp.problem()
x = xp.var(lb=0, ub=10)
Model.addVariable(x)

# Define the objective and solve
test_objective = 5*x
Model.setObjective(test_objective)
Model.solve()

Ok, now we solved the problem. Can we check the objective function separately?

In [16]:
test_objective(5)

TypeError: 'xpress.linterm' object is not callable

No, apparently not. But this is neither Xpress nor Python specific. To my knowledge, this does not exist in CPLEX or Gurobi, and it also does not exist in C# for Xpress. But why is the most important feature for me? There are three reasons:

1) **Working with heuristics**: let's say I have a heuristic that tries to find a better solution by changing part of the solution. How can I check whether this heuristic, that does not use the solver, has found a better solution? Ther eis no way to simply evaluate the objective function and check. It is equally not possible to simply check whether the constraints are satisfied.

2) **Testing**: To check whether a constraint behaves as we expect it to (terms added as they should etc.), it is necessary to evaluate a term. Not having this option makes the tests weaker and doesn't even allow you to run certain tests.

3) **Interaction with user interface**: When developing optimization tools, I like to give the user a sense of the optimizer. Now let's say the user makes a manual change to a solution (e.g. it is a decision support system). When there is no "evaluate" possibility, then how can I give feedback to the user whether his/her change resulted in a better solution from the optimizers point of view?

So, solver people, if you are reading this, please consider including in your expression/constraint objects the possibility for evaluation!

## A cumbsy workaround
I mean, you can do a workaround, by defining a separate problem and then fixing the variables there:

In [ ]:
Model = xp.problem()
x = xp.var(lb=0, ub=10)
Model.addVariable(x)

test_objective = 5*x
Model.setObjective(test_objective)

Model.addConstraint(x==5)
Model.solve()

Model.getObjVal()

However, this has quite a bit of overhead and really does not compare well with the `scipy` approach. However, at least for testing, this is actually what I am forced to do at the moment.